# 2: Global Simulation

Here we will look at how to run a global simulation on CPUs using SPECFEM3D GLOBE. This involves preparing the input data and parameters, generating a mesh, and running the solver.

## 2.1: Prepare Data and Parameters

The parameter files for SPECFEM3D GLOBE are found in the `DATA/` folder. In order to set up for a simulation, we need to:

* Provide the CMTSOLUTION for the event we want to simulate.
* Provide names and locations for the stations we want to use.
* Set the relevant simulation parameters in the `Par_file`.

### 2.1.1: Provide the CMTSOLUTION for the event we want to simulate

The format for the source CMTSOLUTION should follow that of the [Global CMT Catalog](https://www.globalcmt.org/). In this tutorial we will simulate the 2004 [Sumatra Earthquake](https://en.wikipedia.org/wiki/2004_Indian_Ocean_earthquake_and_tsunami).

The CMTSOLUTION for this event looks as follows:

```bash
PDE 2004 12 26  0 58 50.00   3.3000   95.7800  10.0 8.9 8.9 122604A 
event name:     122604A        
time shift:    138.9600
half duration:  95.0000
latitude:        3.0900
longitude:      94.2600
depth:          28.6100
Mrr:       1.040000e+29
Mtt:      -4.270000e+28
Mpp:      -6.100000e+28
Mrt:       2.980000e+29
Mrp:      -2.400000e+29
Mtp:       4.260000e+28
```

This CMTSOLUTION can be downloaded to the `DATA/` folder with the following command:

```bash
wget -O DATA/CMTSOLUTION https://bit.ly/2IqcLSw 
```


### 2.1.2: Provide the names and locations for the stations we want to use

The stations we want to use should be put in the `STATIONS` file, according to the following format:

```bash
Station    Network    Latitude(degrees)    Longitude(degrees)    Elevation(m)    Burial(m)
```

For the event in this tutorial we will use the Global Seismographic Network (GSN):

```bash
AAE        IU        9.0292     38.7656    2442.0     0.0
AAK        II       42.6390     74.4940    1645.0    30.0
ABKT       II       37.9304     58.1189     678.0     7.0
ABPO       II      -19.0180     47.2290    1528.0     5.3
ADK        IU       51.8823   -176.6842     130.0     0.0
...
XMAS       IU        2.0448   -157.4457      19.0     1.0
XPF        II       33.6092   -116.4533    1280.0   100.0
XPFO       II       33.6107   -116.4555    1280.0     0.0
YAK        IU       62.0310    129.6805     110.0    14.0
YSS        IU       46.9587    142.7604     148.0     2.0
```

The STATIONS file can be downloaded to the `DATA/` folder with the following command:

```bash
wget -O DATA/STATIONS https://bit.ly/2P4rjs7
```


### 2.1.3: Set the relevant simulation parameters in the `Par_file`

Most of the simulation parameters are set in the `Par_file`. In this tutorial we will only need to modify a subset of the paramaters in this file. For a full description of all the parameters in the `Par_file`, consult the SPECFEM3D GLOBE [manual](https://specfem3d-globe.readthedocs.io/en/latest/).

Modify the `Par_file` as follows:

```bash
# forward or adjoint simulation
SIMULATION_TYPE                 = 1   # set to 1 for forward simulations, 2 for adjoint simulations for sources, and 3 for kernel simulations
NOISE_TOMOGRAPHY                = 0   # flag of noise tomography, three steps (1,2,3). If earthquake simulation, set it to 0.
SAVE_FORWARD                    = .false.   # save last frame of forward simulation or not
  
# number of chunks (1,2,3 or 6)
NCHUNKS                         = 6

...

CENTER_LATITUDE_IN_DEGREES      = 3.d0
CENTER_LONGITUDE_IN_DEGREES     = 95.d0

...

# number of elements at the surface along the two sides of the first chunk
# (must be multiple of 16 and 8 * multiple of NPROC below)
NEX_XI                          = 64
NEX_ETA                         = 64
  
# number of MPI processors along the two sides of the first chunk
NPROC_XI                        = 2
NPROC_ETA                       = 2

...

# 3D model
MODEL                           = s362ani

# parameters describing the Earth model
OCEANS                          = .true.
ELLIPTICITY                     = .true.
TOPOGRAPHY                      = .true.
GRAVITY                         = .true.
ROTATION                        = .true.
ATTENUATION                     = .true.

# absorbing boundary conditions for a regional simulation
ABSORBING_CONDITIONS            = .false.

# record length in minutes
RECORD_LENGTH_IN_MINUTES        = 10.0d0

...

# path to store the local database files on each node
LOCAL_PATH                      = ./DATABASES_MPI
 
...

# output format for the seismograms (one can use either or all of the three formats)
OUTPUT_SEISMOS_ASCII_TEXT       = .true.
```

Since we have modified the `Par_file`, let's go ahead and recompile SPECFEM:

```bash
make clean && make all
```

This concludes the preparation of the data and simulation parameters, and we are ready to move on to generating the model mesh.

## 2.2: Generate Mesh for Global Simulation

The model mesh can be generated with the `xmeshfem3D` executable. In order to do this, we need to submit a job to the cluster's job scheduler.

The Princeton clusters use the SLURM scheduler to manage their resources. More information about SLURM can be found in the official [documentation](https://slurm.schedmd.com/).

To submit the job, we write a submission script that requests the amount of resources we need for the job and how to run the job. For this job we need 24 cores, since we have 6 chunks and 4 cores per chunk. TigerCPU has 40 cores per node, so we only need to ask for one node. The resulting submission script is the following:

```bash
#!/bin/bash

#SBATCH --nodes 1
#SBATCH --ntasks 24
#SBATCH --time 00:30:00

# load modules
module load intel/18.0
module load openmpi/intel-18.0

# change directory to build
cd /scratch/gpfs/<your_puid>/specfem3d_globe

srun ./bin/xmeshfem3D
```

This submission script can be downloaded with the following command

```bash
wget -O submit_mesher.sh https://bit.ly/2UrStix
```

**NOTE:** You need to modify the path in the submission script by inserting your puid in `cd /scratch/gpfs/<your_puid>/specfem3d_globe`.

This script can then be submitted to the scheduler with the command

```bash
sbatch submit_mesher.sh
```

You can track the progress of the job by typing

```bash
squeue -u <your_puid>
```

The mesher should take a couple of minutes to finish. Once the job is complete, the output can be found in the `OUTPUT_FILES/` and `DATABASES_MPI/` folders. We can check that the mesher ran successfully by inspecting the `OUTPUT_FILES/output_mesher.txt` file.

This file contains information about the mesh, including the number of spectral elements in the mesh, and the size of the time step that will be used when running the solver. The mesher ran successfully if this file ends with `End of mesh generation`.

## 2.3: Run the Solver for Global Simulation

At this point we are ready to run the solver. Before we do this, however, let's go through the following checklist to make sure that we have gone through all the preceeding steps correctly:

* Edit the parameter files.
* Recompile the source code if any paramater in the `Par_file` is changed.
* Generate the model mesh and check the output for success.

With all of these steps covered, we can run the solver.

The solver is run on the cluster with the `xspecfem3D` executable. Similar to what we did for the mesher, we also need to send our solver job to the cluster's job scheduler. The script we use for this looks almost the same

```bash
#!/bin/bash

#SBATCH --nodes 1
#SBATCH --ntasks 24
#SBATCH --time 00:30:00

# load modules
module load intel/18.0
module load openmpi/intel-18.0

# change directory to build
cd /scratch/gpfs/<your_puid>/specfem3d_globe

srun ./bin/xspecfem3D
```

The submission script for the solver can be downloaded with the following command

```bash
wget -O submit_solver.sh https://bit.ly/2UbzqUK
```

**NOTE:** You need to modify the path in the submission script by inserting your puid in `cd /scratch/gpfs/<your_puid>/specfem3d_globe`.

Then we submit the job

```bash
sbatch submit_solver.sh
```

The job can be monitored using `squeue`.

The solver will take longer to finish than the mesher. Once the job is done, the results can be found in the `OUTPUT_FILES/` and `DATABASES_MPI/` folders. Check the `OUTPUT_FILES/output_solver.txt` file to make sure that the solver ran successfully. Upon success, this file will end with `End of the simulation`.